In [4]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import SGDClassifier


dataset_filename = 'C:/GitHub/mbti-type/mbti_1.csv'

# ja possui headers 'type' e 'posts'
mbti = pd.read_csv(dataset_filename)
mbti.head()

#replacing separator |||

for i in range(mbti.shape[0]):
    mbti.loc[i, 'posts'] = mbti.loc[i, 'posts'].replace('|||', ' ')

# TF-IDF scoring

tfidf_model = TfidfVectorizer(smooth_idf=True,
                              sublinear_tf=True,
                              lowercase=True,
                              stop_words='english')

lsa = TruncatedSVD(n_components=50)

results = pd.DataFrame(columns=('Classifier', 'LSA', 'Mean accuracy'))
k = 0

# testing classifiers - Gradient Descent 

classifier = SGDClassifier()

model = Pipeline([
        ('tfidf', tfidf_model),
        ('classifier', classifier)])

accuracy = cross_val_score(estimator=model,
                           X=mbti['posts'],
                           y=mbti['type'],
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1)

results.loc[k] = ['SGDClassifier', False, accuracy.mean()]
k += 1

model = Pipeline([
        ('tfidf', tfidf_model),
        ('lsa', lsa),
        ('classifier', classifier)])

accuracy = cross_val_score(estimator=model,
                           X=mbti['posts'],
                           y=mbti['type'],
                           scoring='accuracy',
                           n_jobs=-1,
                           cv=5)

results.loc[k] = ['SGDClassifier', True, accuracy.mean()]
k += 1

print(results)

C:\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


      Classifier    LSA  Mean accuracy
0  SGDClassifier  False       0.659820
1  SGDClassifier   True       0.577243
